##  Junos PyEZ Operational Tables and Views
Tables and views are another tool for mapping RPC responses into native Python data structures. PyEZ provides several predefined tables and views for common RPCs and also allows users to define their own to extract information from any Junos RPC.

The operational data of a Junos device is the set of state representing the current running conditions of the device. Typically viewing operational data is done using CLI show commands or using operational XML RPCs. This operational data as similar to a database. Databases are organized into a collection of tables, and in a similar way, PyEZ conceptually organizes a Junos device’s operational data into a collection of tables.

In PyEZ, a “table” represents the information that is returned by a particular XML RPC. A PyEZ table is further divided into a list of “items". These items are all of the XML nodes in the RPC output that match a specific XPath expression.

Similar to how a database view selects and presents a subset of fields from a database table, a PyEZ “view” selects and maps a set of fields (XML nodes) from each PyEZ table item into a native Python data structure. Each PyEZ table has at least one view, the default view, for mapping an item’s fields into a native Python data structure. Additional views may be defined, but only the default view is required.

### Pre-packaged  Operational Tables and Views
The location of the *jnpr.junos* module of the PyEZ library is installation-specific. Determine the directory location on your machine by displaying the directory of the *jnpr.junos.__file__* attribute.

In [1]:
import jnpr.junos
import os.path

os.path.dirname(jnpr.junos.__file__)

'/Library/Python/2.7/site-packages/jnpr/junos'

    seanw@Seans-iMac:/Library/Python/2.7/site-packages/jnpr/junos$ tree op/ -P *.yml
    op/
    |-- arp.yml
    |-- bfd.yml
    |-- ccc.yml
    |-- ethernetswitchingtable.yml
    |-- ethport.yml
    |-- fpc.yml
    |-- idpattacks.yml
    |-- intopticdiag.yml
    |-- isis.yml
    |-- l2circuit.yml
    |-- lacp.yml
    |-- ldp.yml
    |-- lldp.yml
    |-- nd.yml
    |-- ospf.yml
    |-- phyport.yml
    |-- routes.yml
    |-- teddb.yml
    |-- vlan.yml
    `-- xcvr.yml

    0 directories, 20 files

In order to use the prepackaged tables and views, list the names of the available tables. 

    seanw@Seans-iMac:/Library/Python/2.7/site-packages/jnpr/junos$ grep Table: op/*.yml
    op/arp.yml:ArpTable:
    op/bfd.yml:BfdSessionTable:
    op/bfd.yml:_BfdSessionClientTable:
    op/ccc.yml:CCCTable:
    op/ethernetswitchingtable.yml:EthernetSwitchingTable:
    op/ethernetswitchingtable.yml:_MacTableEntriesTable:
    op/ethernetswitchingtable.yml:_MacTableInterfacesTable:
    op/ethport.yml:EthPortTable:
    op/fpc.yml:FpcHwTable:
    op/fpc.yml:FpcMiReHwTable:
    op/fpc.yml:FpcInfoTable:
    op/fpc.yml:FpcMiReInfoTable:
    op/idpattacks.yml:IDPAttackTable:
    op/intopticdiag.yml:PhyPortDiagTable:
    op/isis.yml:IsisAdjacencyTable:
    op/isis.yml:_IsisAdjacencyLogTable:
    op/l2circuit.yml:L2CircuitConnectionTable:
    op/lacp.yml:LacpPortTable:
    op/lacp.yml:_LacpPortStateTable:
    op/lacp.yml:_LacpPortProtoTable:
    op/ldp.yml:LdpNeighborTable:
    op/ldp.yml:_LdpNeighborHelloFlagsTable:
    op/ldp.yml:_LdpNeighborTypesTable:
    op/lldp.yml:LLDPNeighborTable:
    op/nd.yml:NdTable:
    op/ospf.yml:OspfNeighborTable:
    op/ospf.yml:OspfInterfaceTable:
    op/phyport.yml:PhyPortTable:
    op/phyport.yml:PhyPortStatsTable:
    op/phyport.yml:PhyPortErrorTable:
    op/routes.yml:RouteTable:
    op/routes.yml:RouteSummaryTable:
    op/routes.yml:_rspTable:
    op/teddb.yml:TedTable:
    op/teddb.yml:_linkTable:
    op/teddb.yml:TedSummaryTable:
    op/vlan.yml:VlanTable:
    op/xcvr.yml:XcvrTable:

In [2]:
from jnpr.junos import Device

vsrx1 = Device(host='vsrx1')
vsrx1.open()

Device(vsrx1)

The first step in using one of these tables is to import the appropriate Python class. The Python class name is the same as the table name defined in the YAML file. Create an empty table instance by passing the device instance variable (vsrx1) as an argument to the class constructor **ArpTable()**. The *arp_table* instance variable can now be populated with all table items by invoking the **get()** instance method. The **get()** method executes an RPC and uses the results to populate the *arp_table* instance with five (5) items.

In [3]:
from jnpr.junos.op.arp import ArpTable

arp_table = ArpTable(vsrx1)
arp_table.get()

ArpTable:vsrx1: 5 items

An alternative way of creating and populating the table is to bind the table as an attribute of the device instance variable. The **get()** method is then invoked on this bound attribute. Storing the *arp_table* as an attribute of the device instance variable is a convenient notation when you’re maintaining a table for multiple devices.

In [4]:
from jnpr.junos.op.arp import ArpTable

vsrx1.bind(arp_table = ArpTable)
vsrx1.arp_table.get()

ArpTable:vsrx1: 5 items

When the **get()** method is invoked, it executes a corresponding XML RPC defined in the table’s YAML file. The **get()** method can be passed the same parameters as if you were invoking the RPC in PyEZ (see: RPC Parameters). The *get-arp-table-information* RPC supports an **`<interface>`** argument to limit the response to ARP entries on a specific interface. Invoking a table’s **get()** method always updates the table’s items by executing an XML RPC against the Junos device and receiving the RPC’s response. 

**NOTE:** Normalization of table and view data is enabled by default.

In [5]:
arp_table.get(interface='ge-0/0/0.1')

ArpTable:vsrx1: 1 items

In [6]:
from pprint import pprint
pprint(arp_table.items())

[('00:0c:29:70:5c:47',
  [('interface_name', 'ge-0/0/0.1'),
   ('ip_address', '20.20.0.1'),
   ('mac_address', '00:0c:29:70:5c:47')])]


Each *jnpr.junos.factory.OpTable* object operates similarly to a Python *OrderedDict* of view objects. The following example refreshes the arp_table instance with all ARP table entries and then shows arp_table’s type is *jnpr.junos.factory.OpTable.ArpTable*, which is a subclass of *jnpr.junos.factory.OpTable*. Like with a dictionary, the keys and values of arp_table are accessed using the **items()** method.

In [7]:
arp_table.get()

ArpTable:vsrx1: 5 items

In [8]:
type(arp_table)

jnpr.junos.factory.OpTable.ArpTable

In [9]:
pprint(arp_table.items())

[('00:0c:29:70:5c:47',
  [('interface_name', 'ge-0/0/0.1'),
   ('ip_address', '20.20.0.1'),
   ('mac_address', '00:0c:29:70:5c:47')]),
 ('00:0c:29:70:5c:51',
  [('interface_name', 'ge-0/0/1.2'),
   ('ip_address', '20.20.0.5'),
   ('mac_address', '00:0c:29:70:5c:51')]),
 ('fe:00:00:00:00:04',
  [('interface_name', 'em0.0'),
   ('ip_address', '128.0.0.16'),
   ('mac_address', 'fe:00:00:00:00:04')]),
 ('00:50:56:c0:00:08',
  [('interface_name', 'fxp0.0'),
   ('ip_address', '172.16.108.1'),
   ('mac_address', '00:50:56:c0:00:08')]),
 ('aa:bb:cc:dd:ee:ff',
  [('interface_name', 'em1.32768'),
   ('ip_address', '192.168.1.1'),
   ('mac_address', 'aa:bb:cc:dd:ee:ff')])]


Because the table operates similarly to an *OrderedDict*, the individual items (which are *jnpr.junos.factory.View.ArpView* objects) can be accessed by either position or key. Remember an *OrderedDict* is a dictionary subclass that remembers the order in which its contents are added. Individual values within a view item can be accessed with two-level references using either an index or a key value for the outer reference.

In [10]:
type(arp_table[0])

jnpr.junos.factory.View.ArpView

In [11]:
pprint(arp_table[0].items())

[('interface_name', 'ge-0/0/0.1'),
 ('ip_address', '20.20.0.1'),
 ('mac_address', '00:0c:29:70:5c:47')]


In [12]:
pprint(arp_table['00:0c:29:70:5c:47'].items())

[('interface_name', 'ge-0/0/0.1'),
 ('ip_address', '20.20.0.1'),
 ('mac_address', '00:0c:29:70:5c:47')]


In [13]:
arp_item = arp_table[0]
arp_item.ip_address

'20.20.0.1'

In [14]:
arp_item.interface_name

'ge-0/0/0.1'

In [15]:
arp_item.mac_address

'00:0c:29:70:5c:47'

In [16]:
arp_item.keys()

['interface_name', 'ip_address', 'mac_address']

In [17]:
arp_item.items()

[('interface_name', 'ge-0/0/0.1'),
 ('ip_address', '20.20.0.1'),
 ('mac_address', '00:0c:29:70:5c:47')]

In addition, every view object has two special properties, *name* and *T*. The name property provides the view’s unique name, or key, within the table. The *T* property is a reference back to the associated table containing the view.

In [18]:
arp_item.name

'00:0c:29:70:5c:47'

In [19]:
arp_item.T

ArpTable:vsrx1: 5 items

In [20]:
vsrx1.close()

### Existing ArpTable and ArpView Definition
PyEZ tables and views are defined in .yml files using the YAML format. PyEZ table and view definitions use a subset of the full YAML syntax. They primarily use associative arrays with multiple levels of hierarchy. Values are typically string data types. Here’s an example using the arp.yml file, which defines the ArpTable and ArpView classes used in the previous section.

    seanw@vsrx1> show arp | display xml rpc 
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <rpc>
            <get-arp-table-information>
            </get-arp-table-information>
        </rpc>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>

    seanw@vsrx1> show arp | display xml        
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <arp-table-information xmlns="http://xml.juniper.net/junos/15.1X49/junos-arp" junos:style="normal">
            <arp-table-entry>
                <mac-address>00:0c:29:6a:f7:a0</mac-address>
                <ip-address>20.20.0.1</ip-address>
                <hostname>20.20.0.1</hostname>
                <interface-name>ge-0/0/0.1</interface-name>
                <arp-table-entry-flags>
                    <none/>
                </arp-table-entry-flags>
            </arp-table-entry>
            <arp-table-entry>
                <mac-address>00:0c:29:6a:f7:c8</mac-address>
                <ip-address>20.20.0.5</ip-address>
                <hostname>20.20.0.5</hostname>
                <interface-name>ge-0/0/1.2</interface-name>
                <arp-table-entry-flags>
                    <none/>
                </arp-table-entry-flags>
            </arp-table-entry>
            <arp-table-entry>
                <mac-address>fe:00:00:00:00:04</mac-address>
                <ip-address>128.0.0.16</ip-address>
                <hostname>128.0.0.16</hostname>
                <interface-name>em0.0</interface-name>
                <arp-table-entry-flags>
                    <none/>
                </arp-table-entry-flags>
            </arp-table-entry>
            <arp-table-entry>
                <mac-address>aa:bb:cc:dd:ee:ff</mac-address>
                <ip-address>192.168.1.1</ip-address>
                <hostname>192.168.1.1</hostname>
                <interface-name>em1.32768</interface-name>
                <arp-table-entry-flags>
                    <none/>
                </arp-table-entry-flags>
            </arp-table-entry>
            <arp-table-entry>
                <mac-address>00:50:56:c0:00:08</mac-address>
                <ip-address>192.168.12.1</ip-address>
                <hostname>192.168.12.1</hostname>
                <interface-name>fxp0.0</interface-name>
                <arp-table-entry-flags>
                    <none/>
                </arp-table-entry-flags>
            </arp-table-entry>
            <arp-entry-count>5</arp-entry-count>
        </arp-table-information>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>

In the *ArpTable* definition:

* *rpc* defines the Junos XML RPC (get-arp-table-information) which is invoked to retrieve the table’s item data.
* *item* is an XPath expression used to select each table item from the RPC response.
* *key* is the XML element within each table item and the value becomes the key used to access each item within the native Python data structure.
* *view* is the name of the default view used to map a table item into a native Python data structure.

In the *ArpView* definition contains *fields*  is an associative array used to match XPath expressions to names:

* *mac_address* the corresponding XPath expression is used to set the value of the mac_address key in the native Python view object.
* *ip_address* is the name of the key in the Python view object that points to the XPath value.
* *interface_name* is another key in the Python view object that will pull the value of the corresponding XPath value

    seanw@Seans-iMac:/Library/Python/2.7/site-packages/jnpr/junos$ cat op/arp.yml
    ---
    ArpTable:
      rpc: get-arp-table-information
      item: arp-table-entry
      key: mac-address
      view: ArpView

    ArpView:
      fields:
        mac_address: mac-address
        ip_address: ip-address
        interface_name: interface-name

### Creating Operational Table and View
Referencing the RPC for show bgp neighbor this overview will cover the definition of a BgpTable and BgpView as well as the structure of the corresponding XML output.

    seanw@vsrx1> show bgp neighbor | display xml rpc 
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <rpc>
            <get-bgp-neighbor-information>
            </get-bgp-neighbor-information>
        </rpc>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>

    seanw@vsrx1> show bgp neighbor | display xml 
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <bgp-information xmlns="http://xml.juniper.net/junos/15.1X49/junos-routing">
            <bgp-peer junos:style="detail">
                <peer-address>20.20.0.1+179</peer-address>
                <peer-as>2087403078</peer-as>
                <local-address>20.20.0.2+55537</local-address>
                <local-as>1234567890</local-as>
                <peer-type>External</peer-type>
                <peer-state>Established</peer-state>
                <peer-flags>Sync</peer-flags>
                <last-state>OpenConfirm</last-state>
                <last-event>RecvKeepAlive</last-event>
                <last-error>None</last-error>
                <bgp-option-information xmlns="http://xml.juniper.net/junos/15.1X49/junos-routing">
                    <export-policy>
                        to-isp2
                    </export-policy>
                    <import-policy>
                        fix-next-hop
                    </import-policy>
                    <bgp-options>Preference LogUpDown AddressFamily PeerAS Multipath Refresh</bgp-options>
                    <bgp-options2>AcceptRemoteNextHop</bgp-options2>
                    <bgp-options-extended></bgp-options-extended>
                    <address-families>inet-unicast inet6-unicast</address-families>
                    <holdtime>90</holdtime>
                    <preference>170</preference>
                </bgp-option-information>
                <flap-count>0</flap-count>
                <peer-id>10.0.255.1</peer-id>
                <local-id>172.27.255.1</local-id>
                <active-holdtime>90</active-holdtime>
                <keepalive-interval>30</keepalive-interval>
                <group-index>0</group-index>
                <peer-index>0</peer-index>
                <bgp-bfd>
                    <bfd-configuration-state>disabled</bfd-configuration-state>
                    <bfd-operational-state>down</bfd-operational-state>
                </bgp-bfd>
                <local-interface-name>ge-0/0/0.1</local-interface-name>
                <local-interface-index>526</local-interface-index>
                <peer-restart-nlri-configured>inet-unicast inet6-unicast</peer-restart-nlri-configured>
                <nlri-type-peer>inet-unicast inet6-unicast</nlri-type-peer>
                <nlri-type-session>inet-unicast inet6-unicast</nlri-type-session>
                <peer-refresh-capability>2</peer-refresh-capability>
                <peer-stale-route-time-configured>300</peer-stale-route-time-configured>
                <peer-no-restart/>
                <peer-restart-flags-received>Notification</peer-restart-flags-received>
                <peer-restart-nlri-negotiated>inet-unicast inet6-unicast</peer-restart-nlri-negotiated>
                <peer-end-of-rib-received>inet-unicast inet6-unicast</peer-end-of-rib-received>
                <peer-end-of-rib-sent>inet-unicast inet6-unicast</peer-end-of-rib-sent>
                <peer-end-of-rib-scheduled></peer-end-of-rib-scheduled>
                <peer-no-llgr-restarter/>
                <peer-4byte-as-capability-advertised>2087403078</peer-4byte-as-capability-advertised>
                <peer-addpath-not-supported/>
                <bgp-rib junos:style="detail">
                    <name>inet.0</name>
                    <rib-bit>10000</rib-bit>
                    <bgp-rib-state>BGP restart is complete</bgp-rib-state>
                    <send-state>in sync</send-state>
                    <active-prefix-count>7</active-prefix-count>
                    <received-prefix-count>7</received-prefix-count>
                    <accepted-prefix-count>7</accepted-prefix-count>
                    <suppressed-prefix-count>0</suppressed-prefix-count>
                    <advertised-prefix-count>1</advertised-prefix-count>
                </bgp-rib>
                <bgp-rib junos:style="detail">
                    <name>inet6.0</name>
                    <rib-bit>20000</rib-bit>
                    <bgp-rib-state>BGP restart is complete</bgp-rib-state>
                    <send-state>in sync</send-state>
                    <active-prefix-count>31</active-prefix-count>
                    <received-prefix-count>31</received-prefix-count>
                    <accepted-prefix-count>31</accepted-prefix-count>
                    <suppressed-prefix-count>0</suppressed-prefix-count>
                    <advertised-prefix-count>1</advertised-prefix-count>
                </bgp-rib>
                <last-received>21</last-received>
                <last-sent>8</last-sent>
                <last-checked>71</last-checked>
                <input-messages>1704</input-messages>
                <input-updates>30</input-updates>
                <input-refreshes>0</input-refreshes>
                <input-octets>34102</input-octets>
                <output-messages>1691</output-messages>
                <output-updates>4</output-updates>
                <output-refreshes>0</output-refreshes>
                <output-octets>32446</output-octets>
                <bgp-output-queue>
                    <number>0</number>
                    <count>0</count>
                    <table-name>inet.0</table-name>
                    <rib-adv-nlri>inet-unicast</rib-adv-nlri>
                </bgp-output-queue>         
                <bgp-output-queue>
                    <number>1</number>
                    <count>0</count>
                    <table-name>inet6.0</table-name>
                    <rib-adv-nlri>inet6-unicast</rib-adv-nlri>
                </bgp-output-queue>
            </bgp-peer>
            [...snip...]
        </bgp-information>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>

This *BgpTable* will focus on BGP peer specific information by extracting **`<bgp-peer>`** elements into table items. The table is populated by running the *get-bgp-summary-information* RPC.  Each element will have a key **`<peer-address>`**, which is a XPath expression relative to a table item **`<bgp-peer>`**. 

The sole purpose of a view definition is to map values to keys in a Python view object. The value for a given key comes from a corresponding XPath expression. The XPath expression is relative to each table item, and typically selects a single element from the table item. The view name becomes the name of a corresponding Python class. Below, *flap*, *state*, and *time* are the field names that contain XPath expressions that select a single corresponding matching child element from a table item:  **`<flap-count>`**,  **`<peer-state>`**, and **`<peer-as>`** (as well as others). Below also shows the use of a nested table view for the **`<bgp-rib>`** elements.

    seanw@Seans-iMac:/Library/Python/2.7/site-packages/jnpr/junos$ more op/bgp.yml 
    ---
    BgpTable: 
        rpc: get-bgp-neighbor-information
        item: bgp-peer
        key: peer-address
        view: BgpView

    BgpView:
        groups:
            rib: bgp-rib
        fields:
            address: peer-address
            state: peer-state
            type: peer-type
            peer_as: peer-as
            local_as: local-as
            local_interface: local-interface-name
            flap_count: flap-count
            rib: _RibTable

    _RibTable:
        item: bgp-rib
        view: _RibView

    _RibView:
        fields:
            table: name
            active: active-prefix-count
            received: received-prefix-count
            accepted: accepted-prefix-count
            suppressed: suppressed-prefix-count
            advertised: advertised-prefix-count

Create a *bgp.py* module in *jnpr.junos.op* package. It loads the *bgp.yml* and creates  BgpTable and BgpView classes. All the "pythonifier" modules in the *jnpr.junos.op* package have the same code - only the name changes.

    seanw@Seans-iMac:/Library/Python/2.7/site-packages/jnpr/junos$ cat op/bgp.py 
    '''
    Pythonifier for Bgp Table/View
    '''
    from jnpr.junos.factory import loadyaml
    from os.path import splitext
    _YAML_ = splitext(__file__)[0] + '.yml'
    globals().update(loadyaml(_YAML_))

In [21]:
from jnpr.junos import Device
from jnpr.junos.op.bgp import BgpTable
from pprint import pprint

vsrx1 = Device(host='vsrx1')
vsrx1.open()

Device(vsrx1)

In [22]:
bgp_table = BgpTable(vsrx1)
bgp_table.get()

BgpTable:vsrx1: 2 items

In [23]:
pprint(bgp_table.items())

[('20.20.0.1+53343',
  [('rib', _RibTable:vsrx1: 2 items),
   ('state', 'Established'),
   ('flap_count', '0'),
   ('peer_as', '2087403078'),
   ('address', '20.20.0.1+53343'),
   ('type', 'External'),
   ('local_interface', 'ge-0/0/0.1'),
   ('local_as', '1234567890')]),
 ('20.20.0.5+60358',
  [('rib', _RibTable:vsrx1: 2 items),
   ('state', 'Established'),
   ('flap_count', '0'),
   ('peer_as', '2087403078'),
   ('address', '20.20.0.5+60358'),
   ('type', 'External'),
   ('local_interface', 'ge-0/0/1.2'),
   ('local_as', '1234567890')])]


In [24]:
import re

for i in bgp_table:
    print "{0} {1}: is {2} (flap count: {3})".format(i.type, re.sub('\+\d+', '', i.address), i.state, i.flap_count)

External 20.20.0.1: is Established (flap count: 0)
External 20.20.0.5: is Established (flap count: 0)


In [25]:
for item in bgp_table:
    print "Peer address: {0}".format(re.sub('\+\d+', '', item.address))
    for rib in item.rib:
        print "Rib table: {0}, Recieved routes: {1}, Active routes: {2}".format(rib.table, rib.received, rib.active)
    print

Peer address: 20.20.0.1
Rib table: inet.0, Recieved routes: 7, Active routes: 7
Rib table: inet6.0, Recieved routes: 31, Active routes: 31

Peer address: 20.20.0.5
Rib table: inet.0, Recieved routes: 7, Active routes: 7
Rib table: inet6.0, Recieved routes: 31, Active routes: 31



In [26]:
vsrx1.close()